# Relaxed Search(제대로) 가미한 코드

### 5월 30일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 15분... 많으면 20분?(데스크탑 기준...)<br>데이터 양에 따라 차이가 남.</strong>



<strong> 참고로 밑에 스스로 판단해서 parameter 넣어하는 칸이 하나 있음.<br>한번 쭉 밀고 그 셀 채우고 다시 쭉 미셈</strong>
<br>
<strong> 아 그리고 XGboost 는 계속 Warning 이 뜨는데 이건 뭔 짓을 해도 뭐 없앨 수가 없네...</strong>

In [1]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=20
# random 인 max_state 를 설정하기 위한 변수.
full_count=0 # 건들지 말것.
# test size
ts = 0.2
# alpha value (밑에 정규검사 등에서 이용되는 p 값)
alpha = 0.05

In [2]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy import stats
import xgboost as xgb
from scipy.stats import wilcoxon
import scipy.stats
import random
random.seed(42)
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [3]:
dataname = "bundang3years_total2.csv"

In [4]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)

Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [5]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for a in Tree_nums:
    for b in Var_nums:
        RF_temp_RMSE_train=0
        RF_temp_RMSE_test=0
        RF_temp_MAPE_train=0
        RF_temp_MAPE_test=0
        RF_temp_RMSE_diff=0
        for i in range(test_times):
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count)
            RF_model.fit(X_train, Y_train)
        
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')   
            
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_temp_RMSE_train+=RF_RMSE_train
            RF_temp_RMSE_test+=RF_RMSE_test
            RF_temp_MAPE_train+=RF_MAPE_train
            RF_temp_MAPE_test+=RF_MAPE_test
            RF_temp_RMSE_diff+=RF_RMSE_diff
            full_count+=1
            count+=1
            progress = round(count/(x*y*test_times)*100,2)
            print(round(progress), end = ", ")
#             print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
             
        RF_RMSE_train_list.append(RF_temp_RMSE_train/test_times)
        RF_RMSE_test_list.append(RF_temp_RMSE_test/test_times)
        RF_MAPE_train_list.append(RF_temp_MAPE_train/test_times)
        RF_MAPE_test_list.append(RF_temp_MAPE_test/test_times)
        RF_par_diff_list.append(RF_temp_RMSE_diff/test_times)
        RF_par_trees_list.append(a)
        RF_par_var_list.append(b)

for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_par_diff_list.pop(index)
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))
    
    
# RF_min_vars_trees_list
# RF_min_vars_var_list

----progress...----
0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 46, 46, 46, 4

In [6]:
print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count )
        RF_Best_model.fit(X_train, Y_train)

        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(10*5)*100,2)
#         print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))


...testing...


In [7]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("Tree_num : {}, Var_num : {}".format(RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

PARAMETERS FOR BEST RF...
Tree_num : 50, Var_num : 1
Tree_num : 10, Var_num : 3
Tree_num : 100, Var_num : 1
Tree_num : 5, Var_num : 1
Tree_num : 750, Var_num : 1


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,13928.200000,7797.350000,13829.400000,13791.050000,13939.550000
RMSE test Mean,13343.050000,8918.250000,13697.850000,13766.750000,13279.250000
RMSE train Stdev,197.779827,285.567553,346.498975,343.844656,233.373408
RMSE test Stdev,807.299974,1345.735482,1465.991926,1300.842222,965.973596
MAPE train Mean,29.693000,16.351000,29.709500,29.758500,29.711500
MAPE test Mean,29.591500,16.466500,29.938500,29.917500,29.455000
MAPE train Stdev,0.232295,0.248614,0.208386,0.420408,0.266595
MAPE test Stdev,0.825647,0.512672,0.983978,0.942883,0.938821
RMSE diff(%) Mean,7.745764,13.060480,10.348954,9.343651,8.011146
RMSE diff(%) Stdev,4.259988,12.671478,6.637945,6.636586,6.981693


### 2. Support Vector Machine parameters

In [8]:
gamma_nums = [0.00005,0.0001,0.000125,0.00015,0.000175,0.0002,]
C_values= [5000,75000,10000,12500,15000]
x = len(gamma_nums)
y = len(C_values)
count=0

SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for a in gamma_nums:
    for b in C_values:
        SVR_temp_RMSE_train=0
        SVR_temp_RMSE_test=0
        SVR_temp_MAPE_train=0
        SVR_temp_MAPE_test=0
        SVR_temp_RMSE_diff=0
        for i in range(test_times): 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)

            SVR_temp_RMSE_train+=SVR_RMSE_train
            SVR_temp_RMSE_test+=SVR_RMSE_test
            SVR_temp_MAPE_train+=SVR_MAPE_train
            SVR_temp_MAPE_test+=SVR_MAPE_test
            SVR_temp_RMSE_diff+=SVR_RMSE_diff
            
            full_count+=1
            count+=1
            progress = round(count/(x*y*test_times)*100,2)
            print(round(progress), end = ", ")
#             print(" {:.2f}% , case:{} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))            
            
        SVR_RMSE_train_list.append(SVR_RMSE_train)
        SVR_RMSE_test_list.append(SVR_RMSE_test)
        SVR_MAPE_train_list.append(SVR_MAPE_train)
        SVR_MAPE_test_list.append(SVR_MAPE_test)
        SVR_par_diff_list.append(SVR_RMSE_diff)
        SVR_par_gamma_list.append(a)
        SVR_par_cval_list.append(b) 

for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_par_diff_list.pop(index)
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))

----progress...----
0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 

In [9]:
print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
        full_count+=1  
        count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    




...testing...



In [10]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
SVR_best_total_dataframe_rbf

PARAMETERS FOR BEST SVR...
Gamma_size: 5e-05 ,  C_values: 12500
Gamma_size: 0.0002 ,  C_values: 12500
Gamma_size: 0.0002 ,  C_values: 10000
Gamma_size: 5e-05 ,  C_values: 75000
Gamma_size: 0.000175 ,  C_values: 75000


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,6102.600000,5602.250000,5899.400000,4757.000000,4562.800000
RMSE test Mean,5430.000000,5767.650000,5391.350000,5384.700000,4370.000000
RMSE train Stdev,296.787196,731.389149,607.825995,515.549901,453.548300
RMSE test Stdev,1112.821819,2327.960293,2003.568598,1885.650580,1817.897934
MAPE train Mean,6.257000,4.755000,4.904500,4.866000,4.029500
MAPE test Mean,6.701000,5.482500,5.609000,5.377500,4.909500
MAPE train Stdev,0.065429,0.046206,0.033834,0.050537,0.042009
MAPE test Stdev,0.325252,0.199521,0.188465,0.187373,0.205681
RMSE diff(%) Mean,23.697132,43.264712,41.409192,35.500370,43.691546
RMSE diff(%) Stdev,15.329858,22.057118,27.549678,16.921731,23.071727


### 3. XGBoost ... <br>
DAMN...

In [11]:
feattree=[0.2,0.4,0.6]
learnrate=[0.1,0.2,0.3,0.4]
maxdepth=[5,7,9,11]
trees=[50,75,100,250,500]
count=0

XGB_RMSE_train_list =[]
XGB_RMSE_test_list =[]
XGB_MAPE_train_list =[]
XGB_MAPE_test_list =[]
XGB_par_diff_list=[]

XGB_par_feattree_list=[]
XGB_par_learnrate_list=[]
XGB_par_maxdepth_list=[]
XGB_par_trees_list=[]

XGB_min_vars_feattree_list=[]
XGB_min_vars_learnrate_list=[]  
XGB_min_vars_maxdepth_list=[]
XGB_min_vars_trees_list=[] 

print("----progress...----")

for a in feattree:
    for b in learnrate:
        for c in maxdepth:
            for d in trees:
                X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
                xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = a, 
                                              learning_rate = b, max_depth = c, n_estimators = d)
                xg_reg.fit(X_train,Y_train)
                    
                XGB_train_test = xg_reg.predict(X_train).astype('int')
                XGB_prediction_test = xg_reg.predict(X_test).astype('int')
                    
                XGB_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_train_test)).astype('int')
                XGB_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_prediction_test)).astype('int')
                XGB_MAPE_train = round(np.mean(np.abs((Y_train - XGB_train_test) / Y_train)) * 100 ,2)
                XGB_MAPE_test = round(np.mean(np.abs((Y_test - XGB_prediction_test) / Y_test)) * 100 ,2)
                XGB_RMSE_diff = np.abs((XGB_RMSE_test-XGB_RMSE_train)/XGB_RMSE_test *100)

                full_count+=1
                count+=1
                    
#                 progress = round(count/(len(feattree)*len(learnrate)*len(maxdepth)*len(trees))*100,2)
#                 print(round(progress), end = ", ")
                
#                 print("{} {} {} {} \n{} {} {} {} {}".
#                     format(a, b, c,d,
#                         XGB_RMSE_train,XGB_RMSE_test,XGB_MAPE_train,XGB_MAPE_test,round(XGB_RMSE_diff,2) ))
                
                
                XGB_RMSE_train_list.append(XGB_RMSE_train)
                XGB_RMSE_test_list.append(XGB_RMSE_test)
                XGB_MAPE_train_list.append(XGB_MAPE_train)
                XGB_MAPE_test_list.append(XGB_MAPE_test)
                XGB_par_diff_list.append(XGB_RMSE_diff)
                
                XGB_par_feattree_list.append(a)
                XGB_par_learnrate_list.append(b)
                XGB_par_maxdepth_list.append(c)
                XGB_par_trees_list.append(d)

sp_count=0
while sp_count < 5:
    val = min(XGB_par_diff_list)
    index = XGB_par_diff_list.index(val)
    XGB_par_diff_list.pop(index)
    if((XGB_MAPE_train_list[index])<=15 and (XGB_MAPE_test_list[index])<=15):
        XGB_min_vars_feattree_list.append(XGB_par_feattree_list.pop(index))
        XGB_min_vars_learnrate_list.append(XGB_par_learnrate_list.pop(index))
        XGB_min_vars_maxdepth_list.append(XGB_par_maxdepth_list.pop(index))
        XGB_min_vars_trees_list.append(XGB_par_trees_list.pop(index))
        sp_count+=1

----progress...----
[16:26:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/

In [12]:
print("PARAMETERS FOR BEST XGB...")
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

print("...testing...")

progress=0
count=0
XGB_Best_RMSE_train_list=[]
XGB_Best_RMSE_test_list=[]
XGB_Best_MAPE_train_list=[]
XGB_Best_MAPE_test_list=[]
XGB_Best_RMSE_diff_list=[]

XGB_Best_total_data=[]

for i in range(5):
    temp_XGB_Best_RMSE_train_list=[]
    temp_XGB_Best_RMSE_test_list=[]
    temp_XGB_Best_MAPE_train_list=[]
    temp_XGB_Best_MAPE_test_list=[]
    temp_XGB_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = XGB_min_vars_feattree_list[i]
        b = XGB_min_vars_learnrate_list[i]
        c = XGB_min_vars_maxdepth_list[i]
        d = XGB_min_vars_trees_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        xg_Best_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = a, 
                                learning_rate = b, max_depth = c, n_estimators = d)
        
        xg_Best_reg.fit(X_train,Y_train)
                    
        XGB_Best_train_test = xg_Best_reg.predict(X_train).astype('int')
        XGB_Best_prediction_test = xg_Best_reg.predict(X_test).astype('int')
                    
        XGB_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_Best_train_test)).astype('int')
        XGB_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_Best_prediction_test)).astype('int')

        XGB_Best_RMSE_train = np.sqrt(mean_squared_error(Y_train, XGB_Best_train_test)).astype('int')
        XGB_Best_MAPE_train = round(np.mean(np.abs((Y_train - XGB_Best_train_test) / Y_train)) * 100 ,2)

        XGB_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_Best_prediction_test)).astype('int')
        XGB_Best_MAPE_test = round(np.mean(np.abs((Y_test - XGB_Best_prediction_test) / Y_test)) * 100 ,2)

        XGB_Best_RMSE_diff = np.abs((XGB_Best_RMSE_test-XGB_Best_RMSE_train)/XGB_Best_RMSE_test *100)
        
        temp_XGB_Best_RMSE_train_list.append(XGB_Best_RMSE_train)
        temp_XGB_Best_RMSE_test_list.append(XGB_Best_RMSE_test)
        temp_XGB_Best_MAPE_train_list.append(XGB_Best_MAPE_train)
        temp_XGB_Best_MAPE_test_list.append(XGB_Best_MAPE_test)
        temp_XGB_Best_RMSE_diff_list.append(XGB_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(5*test_times)*100,2)

    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_train_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_test_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_train_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_test_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_MAPE_train_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_MAPE_test_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_MAPE_train_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_MAPE_test_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_diff_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_diff_list))


PARAMETERS FOR BEST XGB...
 feature % : 0.4, learn_rate : 0.1, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.3, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.3, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
...testing...
[16:30:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:30:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:30:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[16:30:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:

In [13]:
print("PARAMETERS FOR BEST XGB...")
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

XGB1=[]
XGB2=[]
XGB3=[]
XGB4=[]
XGB5=[]

for i in range(50):
    k=i//10
    if k==0:
        XGB1.append(XGB_Best_total_data[i])
    if k==1:
        XGB2.append(XGB_Best_total_data[i])
    if k==2:
        XGB3.append(XGB_Best_total_data[i])
    if k==3:
        XGB4.append(XGB_Best_total_data[i])
    if k==4:
        XGB5.append(XGB_Best_total_data[i])
    
XGB_s1 = pd.Series(XGB1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

XGB_s2 = pd.Series(XGB2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
XGB_s3 = pd.Series(XGB3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
XGB_s4 = pd.Series(XGB4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
XGB_s5 = pd.Series(XGB5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')


XGB_best_total_dataframe = pd.concat([XGB_s1,XGB_s2,XGB_s3,XGB_s4,XGB_s5],axis=1)
print("PARAMETERS FOR BEST XGB...")
XGB_best_total_dataframe

PARAMETERS FOR BEST XGB...
 feature % : 0.4, learn_rate : 0.1, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.3, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.3, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
PARAMETERS FOR BEST XGB...


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3785.350000,6320.900000,3800.450000,2778.400000,5120.500000
RMSE test Mean,6034.300000,7172.700000,5433.000000,5113.700000,7181.000000
RMSE train Stdev,142.810460,130.743604,105.302172,88.308776,105.419400
RMSE test Stdev,1776.488674,1162.930355,1468.480848,1632.906430,1161.768350
MAPE train Mean,7.346000,11.781000,7.694500,5.878000,9.747500
MAPE test Mean,8.207500,12.957500,8.837000,7.220500,10.868500
MAPE train Stdev,0.137128,0.115754,0.124799,0.134410,0.092514
MAPE test Stdev,0.318542,0.421401,0.335218,0.241733,0.349489
RMSE diff(%) Mean,31.833060,12.030794,25.767716,40.607869,26.732414
RMSE diff(%) Stdev,18.682465,11.614269,17.011991,16.390282,12.344874


# COMPARING ALL FOUR IN ONE

In [14]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("Tree_num : {}, Var_num : {}".format(RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))
RF_best_total_dataframe

PARAMETERS FOR BEST RF...
Tree_num : 50, Var_num : 1
Tree_num : 10, Var_num : 3
Tree_num : 100, Var_num : 1
Tree_num : 5, Var_num : 1
Tree_num : 750, Var_num : 1


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,13928.200000,7797.350000,13829.400000,13791.050000,13939.550000
RMSE test Mean,13343.050000,8918.250000,13697.850000,13766.750000,13279.250000
RMSE train Stdev,197.779827,285.567553,346.498975,343.844656,233.373408
RMSE test Stdev,807.299974,1345.735482,1465.991926,1300.842222,965.973596
MAPE train Mean,29.693000,16.351000,29.709500,29.758500,29.711500
MAPE test Mean,29.591500,16.466500,29.938500,29.917500,29.455000
MAPE train Stdev,0.232295,0.248614,0.208386,0.420408,0.266595
MAPE test Stdev,0.825647,0.512672,0.983978,0.942883,0.938821
RMSE diff(%) Mean,7.745764,13.060480,10.348954,9.343651,8.011146
RMSE diff(%) Stdev,4.259988,12.671478,6.637945,6.636586,6.981693


In [15]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))
SVR_best_total_dataframe_rbf

PARAMETERS FOR BEST SVR...
Gamma_size: 5e-05 ,  C_values: 12500
Gamma_size: 0.0002 ,  C_values: 12500
Gamma_size: 0.0002 ,  C_values: 10000
Gamma_size: 5e-05 ,  C_values: 75000
Gamma_size: 0.000175 ,  C_values: 75000


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,6102.600000,5602.250000,5899.400000,4757.000000,4562.800000
RMSE test Mean,5430.000000,5767.650000,5391.350000,5384.700000,4370.000000
RMSE train Stdev,296.787196,731.389149,607.825995,515.549901,453.548300
RMSE test Stdev,1112.821819,2327.960293,2003.568598,1885.650580,1817.897934
MAPE train Mean,6.257000,4.755000,4.904500,4.866000,4.029500
MAPE test Mean,6.701000,5.482500,5.609000,5.377500,4.909500
MAPE train Stdev,0.065429,0.046206,0.033834,0.050537,0.042009
MAPE test Stdev,0.325252,0.199521,0.188465,0.187373,0.205681
RMSE diff(%) Mean,23.697132,43.264712,41.409192,35.500370,43.691546
RMSE diff(%) Stdev,15.329858,22.057118,27.549678,16.921731,23.071727


In [16]:
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

XGB_best_total_dataframe

 feature % : 0.4, learn_rate : 0.1, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.3, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.3, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3785.350000,6320.900000,3800.450000,2778.400000,5120.500000
RMSE test Mean,6034.300000,7172.700000,5433.000000,5113.700000,7181.000000
RMSE train Stdev,142.810460,130.743604,105.302172,88.308776,105.419400
RMSE test Stdev,1776.488674,1162.930355,1468.480848,1632.906430,1161.768350
MAPE train Mean,7.346000,11.781000,7.694500,5.878000,9.747500
MAPE test Mean,8.207500,12.957500,8.837000,7.220500,10.868500
MAPE train Stdev,0.137128,0.115754,0.124799,0.134410,0.092514
MAPE test Stdev,0.318542,0.421401,0.335218,0.241733,0.349489
RMSE diff(%) Mean,31.833060,12.030794,25.767716,40.607869,26.732414
RMSE diff(%) Stdev,18.682465,11.614269,17.011991,16.390282,12.344874


아래 셀은 값들을 직접 입력할것.<br>
위에 표 보고 제일 괜찮을 것 같은거 스스로 판단해서<br>
그 parameter 들을 아래에다 넣자.<br>
그리고 다시 쭉 밀면 된다.<br>

In [5]:
RF_trees= 10
RF_depth= 3
SVR_gamma= 0.000175
SVR_c= 75000
XGB_colsample=0.2
XGB_learn_rate=0.3
XGB_max_depth=5
XGB_trees=75

In [6]:
final_RF_RMSE_train=[]
final_RF_RMSE_test=[]
final_RF_MAPE_train=[]
final_RF_MAPE_test=[]
final_RF_RMSE_diff=[]

final_SVR_RMSE_train=[]
final_SVR_RMSE_test=[]
final_SVR_MAPE_train=[]
final_SVR_MAPE_test=[]
final_SVR_RMSE_diff=[]

final_XGB_RMSE_train=[]
final_XGB_RMSE_test=[]
final_XGB_MAPE_train=[]
final_XGB_MAPE_test=[]
final_XGB_RMSE_diff=[]

final_LR_RMSE_train=[]
final_LR_RMSE_test=[]
final_LR_MAPE_train=[]
final_LR_MAPE_test=[]
final_LR_RMSE_diff=[]

count=0
for i in range(test_times):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
    
    # RF
    
    RF_model = RandomForestRegressor(n_estimators=RF_trees, max_depth=RF_depth,random_state=full_count)
    RF_model.fit(X_train, Y_train)      
    RF_prediction_test = RF_model.predict(X_test).astype('int')  
    RF_train_test = RF_model.predict(X_train).astype('int') 
    
    RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
    RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
    RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2) 
    RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)        
    RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)      
    
    final_RF_RMSE_train.append(RF_RMSE_train)
    final_RF_RMSE_test.append(RF_RMSE_test)
    final_RF_MAPE_train.append(RF_MAPE_train)
    final_RF_MAPE_test.append(RF_MAPE_test)
    final_RF_RMSE_diff.append(RF_RMSE_diff)
    
    full_count+=1
    
    # SVR
    
    SVR_model = svm.SVR(kernel='rbf',gamma=SVR_gamma,C=SVR_c)
    SVR_model.fit(X_train, Y_train)
    SVR_prediction_test = SVR_model.predict(X_test).astype('int')       
    SVR_train_test = SVR_model.predict(X_train).astype('int')
    
    SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
    SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
    SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
    SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
    SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
    final_SVR_RMSE_train.append(SVR_RMSE_train)
    final_SVR_RMSE_test.append(SVR_RMSE_test)
    final_SVR_MAPE_train.append(SVR_MAPE_train)
    final_SVR_MAPE_test.append(SVR_MAPE_test)        
    final_SVR_RMSE_diff.append(SVR_RMSE_diff)
    
    full_count+=1
        
    # XGboost 
        
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = XGB_colsample, learning_rate = XGB_learn_rate, 
                              max_depth = XGB_max_depth, n_estimators = XGB_trees)
    xg_reg.fit(X_train,Y_train)
    
    XGB_train_test = xg_reg.predict(X_train).astype('int')
    XGB_prediction_test = xg_reg.predict(X_test).astype('int')
                    
    XGB_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_train_test)).astype('int')
    XGB_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_prediction_test)).astype('int')
    XGB_MAPE_train = round(np.mean(np.abs((Y_train - XGB_train_test) / Y_train)) * 100 ,2)
    XGB_MAPE_test = round(np.mean(np.abs((Y_test - XGB_prediction_test) / Y_test)) * 100 ,2)
    XGB_RMSE_diff = np.abs((XGB_RMSE_test-XGB_RMSE_train)/XGB_RMSE_test *100)
    
    final_XGB_RMSE_train.append(XGB_RMSE_train)
    final_XGB_RMSE_test.append(XGB_RMSE_test)
    final_XGB_MAPE_train.append(XGB_MAPE_train)
    final_XGB_MAPE_test.append(XGB_MAPE_test)
    final_XGB_RMSE_diff.append(XGB_RMSE_diff)    

    full_count+=1

    # LR
    
    LR_model=LinearRegression()
    LR_model.fit(X_train, Y_train)
    LR_prediction_test = LR_model.predict(X_test).astype('int')
    LR_train_test = LR_model.predict(X_train).astype('int')

    
    LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
    LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')    
    LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
    LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)            
    LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
    
    final_LR_RMSE_train.append(LR_RMSE_train)
    final_LR_RMSE_test.append(LR_RMSE_test)
    final_LR_MAPE_train.append(LR_MAPE_train)
    final_LR_MAPE_test.append(LR_MAPE_test)    
    final_LR_RMSE_diff.append(LR_RMSE_diff)
    
    full_count+=1


[14:47:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[14:47:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[14:47:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[14:47:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[14:47:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[14:47:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:17

In [7]:
print("RMSE values")
RMSE_test_all = pd.concat([pd.Series(final_RF_RMSE_test),pd.Series(final_SVR_RMSE_test),
                           pd.Series(final_XGB_RMSE_test),pd.Series(final_LR_RMSE_test),pd.Series(final_LR_RMSE_train)],axis=1)

RMSE_test_all = RMSE_test_all.rename({'0':'RF'},axis='columns')
RMSE_test_all.columns=['RF','SVR','XGB','LR','train']
RMSE_test_all

RMSE values


,RF,SVR,XGB,LR,train
0,8587,4201,3969,7096,8005
1,7603,3216,3963,7783,7849
2,7568,3707,3945,6893,8051
3,7718,3479,3772,7314,7958
4,10830,8093,7490,9457,7372
5,7711,4693,4019,7087,8008
6,7489,3149,3909,7072,8020
7,9497,3611,5997,7367,7954
8,7238,3406,3711,6665,8101
9,7341,3654,3875,6749,8078


In [8]:
RF_RMSE_mean = mean(RMSE_test_all['RF'])
SVR_RMSE_mean = mean(RMSE_test_all['SVR'])
XGB_RMSE_mean = mean(RMSE_test_all['XGB'])
LR_RMSE_mean = mean(RMSE_test_all['LR'])
print("RF_RMSE_mean : {} \nSVR_RMSE_mean : {} \nXGB_RMSE_mean : {} \nLR_RMSE_mean : {}".
      format(RF_RMSE_mean,SVR_RMSE_mean,XGB_RMSE_mean,LR_RMSE_mean))

print("max: {}".format(max(RF_RMSE_mean,SVR_RMSE_mean,XGB_RMSE_mean,LR_RMSE_mean)))


RF_RMSE_mean : 8300.4 
SVR_RMSE_mean : 4338.45 
XGB_RMSE_mean : 4817.6 
LR_RMSE_mean : 7458.45
max: 8300.4


In [9]:
print("MAPE values")
MAPE_test_all = pd.concat([pd.Series(final_RF_MAPE_test),pd.Series(final_SVR_MAPE_test),
                           pd.Series(final_XGB_MAPE_test),pd.Series(final_LR_MAPE_test),pd.Series(final_LR_MAPE_train)],axis=1)

MAPE_test_all = MAPE_test_all.rename({'0':'RF'},axis='columns')
MAPE_test_all.columns=['RF','SVR','XGB','LR','train']
MAPE_test_all

MAPE values


,RF,SVR,XGB,LR,train
0,16.70,4.78,7.19,16.74,16.40
1,15.42,4.63,7.26,16.83,16.13
2,16.71,5.05,7.26,16.14,16.60
3,16.75,5.00,7.27,16.31,16.40
4,17.12,4.69,6.76,16.23,16.80
5,16.56,5.14,6.50,15.64,16.62
6,17.34,5.09,7.59,16.94,16.58
7,16.61,4.84,6.79,15.89,16.50
8,16.78,4.96,6.93,16.20,16.78
9,15.93,5.03,7.35,15.79,16.73


In [10]:
RF_MAPE_mean = mean(MAPE_test_all['RF'])
SVR_MAPE_mean = mean(MAPE_test_all['SVR'])
XGB_MAPE_mean = mean(MAPE_test_all['XGB'])
LR_MAPE_mean = mean(MAPE_test_all['LR'])
print("RF_MAPE_mean : {} \nSVR_MAPE_mean : {} \nXGB_MAPE_mean : {} \nLR_MAPE_mean : {}".
      format(RF_MAPE_mean,SVR_MAPE_mean,XGB_MAPE_mean,LR_MAPE_mean))

print("max: {}".format(max(RF_MAPE_mean,SVR_MAPE_mean,XGB_MAPE_mean,LR_MAPE_mean)))

RF_MAPE_mean : 16.4125 
SVR_MAPE_mean : 4.958 
XGB_MAPE_mean : 7.212499999999999 
LR_MAPE_mean : 16.398999999999997
max: 16.4125


In [11]:
print("diff values")
diff_test_all = pd.concat([pd.Series(final_RF_RMSE_diff),pd.Series(final_SVR_RMSE_diff),
                           pd.Series(final_XGB_RMSE_diff),pd.Series(final_LR_RMSE_diff)],axis=1)

diff_test_all = diff_test_all.rename({'0':'RF'},axis='columns')
diff_test_all.columns=['RF','SVR','XGB','LR']
diff_test_all


diff values


,RF,SVR,XGB,LR
0,5.648073,11.378243,26.883346,12.810034
1,2.301723,49.533582,32.046429,0.848002
2,4.770085,29.188023,28.745247,16.799652
3,3.653796,40.212705,22.826087,8.805031
4,29.252078,54.751019,64.205607,22.047161
5,3.760861,0.958875,30.977855,12.995626
6,6.449459,56.208320,28.037861,13.404977
7,17.984627,32.262531,55.110889,7.967965
8,12.669246,39.812096,24.333064,21.545386
9,8.895246,33.880679,27.122581,19.691806


In [12]:
RF_diff_mean = mean(diff_test_all['RF'])
SVR_diff_mean = mean(diff_test_all['SVR'])
XGB_diff_mean = mean(diff_test_all['XGB'])
LR_diff_mean = mean(diff_test_all['LR'])
print("RF_diff_mean : {} \nSVR_diff_mean : {} \nXGB_diff_mean : {} \nLR_diff_mean : {}".
      format(RF_diff_mean,SVR_diff_mean,XGB_diff_mean,LR_diff_mean))

print("max: {}".format(max(RF_diff_mean,SVR_diff_mean,XGB_diff_mean,LR_diff_mean)))


RF_diff_mean : 12.269722108710816 
SVR_diff_mean : 37.472402645715135 
XGB_diff_mean : 37.02896213262362 
LR_diff_mean : 14.849020317242411
max: 37.472402645715135


In [13]:
# 정규 검정. 오직 하나의 줄, array 를 이용할 것
def Ntest(df):
    stat,p = stats.normaltest(df)
    if p>alpha:
        return False
    else:
        return True
    
# 그냥 4개를 정규검정해서 list 로 True False 반환
def fourNtest(k):
    a = k['RF']
    b = k['SVR']
    c = k['XGB']
    d = k['LR']
    tlist=[]
    tlist.append(Ntest(a))
    tlist.append(Ntest(b))
    tlist.append(Ntest(c))
    tlist.append(Ntest(d))
    return tlist

# T-test. Array 로 시킬 것.
def Ttest(df1,df2):
    s,p=stats.ttest_rel(df1, df2)
    if p>alpha:
        return False
    else:
        return True

# Wilcoxon test. Array 로 정큐분포를 이루지 않을때의 paired t-test
def Wiltest(df1,df2):
    z, p = wilcoxon(df2 - df1)
    print(p)
    if p>alpha:
        return False
    else:
        return True

In [14]:
print(fourNtest(RMSE_test_all))

[False, True, False, True]


In [15]:
print(fourNtest(MAPE_test_all))

[False, False, False, False]


In [16]:
print(fourNtest(diff_test_all))

[False, False, False, False]


그냥 다 정규분포 안 따른다고 생각해도 될듯

In [17]:
# RMSE_test_all['RF']    RMSE_test_all['SVR']    RMSE_test_all['XGB']    RMSE_test_all['LR']
# MAPE_test_all['RF']    MAPE_test_all['SVR']    MAPE_test_all['XGB']    MAPE_test_all['LR']
# diff_test_all['RF']    diff_test_all['SVR']    diff_test_all['XGB']    diff_test_all['LR']



# RMSE 간 차이가 있는지를 확인
print("-----RMSE-----")
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['SVR']))
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['XGB']))
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['LR']))
print(Wiltest(RMSE_test_all['SVR'],RMSE_test_all['XGB']))
print(Wiltest(RMSE_test_all['SVR'],RMSE_test_all['LR']))
print(Wiltest(RMSE_test_all['XGB'],RMSE_test_all['LR']))

# MAPE 간 차이가 있는지를 확인
print("-----MAPE-----")
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['SVR']))
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['XGB']))
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['LR']))
print(Wiltest(MAPE_test_all['SVR'],MAPE_test_all['XGB']))
print(Wiltest(MAPE_test_all['SVR'],MAPE_test_all['LR']))
print(Wiltest(MAPE_test_all['XGB'],MAPE_test_all['LR']))

# diff 간 차이가 있는지를 확인
print("-----diff-----")
print(Wiltest(diff_test_all['RF'],diff_test_all['SVR']))
print(Wiltest(diff_test_all['RF'],diff_test_all['XGB']))
print(Wiltest(diff_test_all['RF'],diff_test_all['LR']))
print(Wiltest(diff_test_all['SVR'],diff_test_all['XGB']))
print(Wiltest(diff_test_all['SVR'],diff_test_all['LR']))
print(Wiltest(diff_test_all['XGB'],diff_test_all['LR']))

-----RMSE-----
1.9073486328125e-06
True
1.9073486328125e-06
True
3.814697265625e-06
True
0.13272666931152344
False
1.9073486328125e-06
True
1.9073486328125e-06
True
-----MAPE-----
1.9073486328125e-06
True
1.9073486328125e-06
True
0.7561664581298828
False
1.9073486328125e-06
True
1.9073486328125e-06
True
1.9073486328125e-06
True
-----diff-----
3.814697265625e-06
True
1.33514404296875e-05
True
0.16495704650878906
False
0.8408222198486328
False
2.6702880859375e-05
True
0.000209808349609375
True


결과: RMSE 든 MAPE 든 모델 간 성능 차이가 확실하다.